In [61]:
pip install numpy==2.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


In [9]:
import yfinance as yf
import pandas as pd
import talib
import numpy as np

# 1. Fetch intraday data for INFY.NS (5-minute bars, last 5 days)
symbol = "INFY.NS"
data = yf.download(symbol,
                   period="5d",
                   interval="5m",
                   auto_adjust=True,
                   progress=False,
                   multi_level_index=False)


# Ensure data is sorted chronologically
data = data.sort_index()

# 2. Identify Engulfing patterns
#   +100 for bullish engulfing, -100 for bearish
data["engulf"] = talib.CDLENGULFING(
    data["Open"], data["High"], data["Low"], data["Close"]
)

data

# 3. Generate signals and backtest
trades = []
for i in range(len(data) - 1):
    signal = data["engulf"].iloc[i]
    if signal == 100:    # Bullish engulfing → go long next bar open → exit at next bar close
        entry_price = data["Open"].iloc[i+1]
        exit_price  = data["Close"].iloc[i+1]
        ret = (exit_price - entry_price) / entry_price
        trades.append({"type":"long", "entry": entry_price, "exit": exit_price, "return": ret})
    elif signal == -100: # Bearish engulfing → go short next bar open → exit at next bar close
        entry_price = data["Open"].iloc[i+1]
        exit_price  = data["Close"].iloc[i+1]
        ret = (entry_price - exit_price) / entry_price
        trades.append({"type":"short","entry": entry_price, "exit": exit_price, "return": ret})

# 4. Build DataFrame of trades
trades_df = pd.DataFrame(trades)

# 5. Compute metrics
total_trades = len(trades_df)
wins         = trades_df["return"] > 0
win_rate     = wins.mean() * 100 if total_trades>0 else np.nan
avg_return   = trades_df["return"].mean() * 100 if total_trades>0 else np.nan
gross_profit = trades_df.loc[trades_df["return"]>0, "return"].sum()
gross_loss   = -trades_df.loc[trades_df["return"]<0, "return"].sum()
profit_factor= gross_profit / gross_loss if gross_loss>0 else np.nan

# 6. Print summary
print(f"Backtest of CDLENGULFING on {symbol} (5m bars, last 5d)")
print(f"Total trades      : {total_trades}")
print(f"Win rate          : {win_rate:.2f}%")
print(f"Average return    : {avg_return:.2f}% per trade")
print(f"Gross profit      : {gross_profit*100:.2f}%")
print(f"Gross loss        : {-gross_loss*100:.2f}%")
print(f"Profit factor     : {profit_factor:.2f}")

# 7. Print trade-level P/L
print("\nTrade details:")
print(trades_df.assign(
    return_pct=lambda df: (df["return"] * 100).round(2)
)[["type", "entry", "exit", "return_pct"]].to_string(index=False))


Backtest of CDLENGULFING on INFY.NS (5m bars, last 5d)
Total trades      : 22
Win rate          : 68.18%
Average return    : 0.09% per trade
Gross profit      : 2.48%
Gross loss        : -0.55%
Profit factor     : 4.49

Trade details:
 type       entry        exit  return_pct
short 1579.000000 1578.800049        0.01
 long 1583.500000 1583.900024        0.03
short 1572.000000 1570.900024        0.07
short 1568.800049 1570.000000       -0.08
 long 1570.400024 1575.099976        0.30
 long 1576.500000 1577.300049        0.05
short 1578.800049 1579.599976       -0.05
 long 1579.599976 1579.599976        0.00
short 1586.000000 1582.599976        0.21
short 1573.199951 1561.300049        0.76
 long 1552.900024 1554.000000        0.07
 long 1545.099976 1548.800049        0.24
short 1542.000000 1539.599976        0.16
 long 1532.500000 1536.699951        0.27
 long 1533.199951 1535.199951        0.13
short 1531.300049 1532.300049       -0.07
short 1530.400024 1529.500000        0.06
short 152

In [10]:
import yfinance as yf
import pandas as pd
import talib
import vectorbt as vbt
import numpy as np

# Download data
data = yf.download("INFY.NS", period="5d", interval="5m", auto_adjust=True, progress=False, multi_level_index=False)
data.dropna(inplace=True)

# Calculate indicators
engulf = talib.CDLENGULFING(data['Open'], data['High'], data['Low'], data['Close'])
atr = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

# Create entry signals
entries = engulf == 100
pattern_exits = engulf == -100

# Create ATR-based trailing stop
atr_multiplier = 2.0  # 2x ATR for stop distance

# Calculate dynamic stop loss levels
stop_distance = atr * atr_multiplier
stop_distance = stop_distance.fillna(method='ffill').fillna(0)

# Custom fee function (same as before)
def groww_fees(size, price, direction):
    value = size * price
    brokerage = min(20.0, 0.0005 * value)
    stt = 0.00025 * value if direction == 'sell' else 0.0
    exchange = 0.0000297 * value
    sebi = 0.000001 * value
    stamp = 0.00003 * value if direction == 'buy' else 0.0
    gst = 0.18 * (brokerage + exchange + sebi)
    total_charges = brokerage + stt + exchange + sebi + stamp + gst
    return total_charges / value

# Calculate fee rate
avg_price = data['Close'].mean()
avg_size = 100
buy_fee_rate = groww_fees(avg_size, avg_price, 'buy')
sell_fee_rate = groww_fees(avg_size, avg_price, 'sell')
avg_fee_rate = (buy_fee_rate + sell_fee_rate) / 2

print(f"Estimated total fee rate: {avg_fee_rate*100:.3f}%")

# Run backtest with ATR trailing stop
portfolio = vbt.Portfolio.from_signals(
    data['Close'],
    entries=entries,
    exits=pattern_exits,
    init_cash=10000000,
    fees=avg_fee_rate,
    freq='5T',
    # ATR-based trailing stop
    sl_stop=stop_distance / data['Close'],  # Dynamic stop as percentage
    sl_trail=True
)

# Display results
stats = portfolio.stats()
trades = portfolio.trades.records_readable

print(f"\nVectorBT CDLENGULFING with ATR Trailing Stop Results")
print(f"===================================================")
print(f"Initial Capital: ₹{100000:,}")
print(f"Final Value: ₹{portfolio.value().iloc[-1]:,.2f}")
print(f"Total Return: {stats['Total Return [%]']:.2f}%")
print(f"Total Trades: {len(trades)}")
print(f"Win Rate: {stats['Win Rate [%]']:.2f}%") 
print(f"Max Drawdown: {stats['Max Drawdown [%]']:.2f}%")
print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
print(f"Profit Factor: {stats['Profit Factor']:.2f}")
print(f"Average ATR Stop Distance: {(stop_distance/data['Close']*100).mean():.2f}%")

if len(trades) > 0:
    print(f"\nTrade Details:")
    print(trades[['Direction', 'Avg Entry Price', 'Avg Exit Price', 'PnL', 'Return', 'Status']].to_string(index=False))


Estimated total fee rate: 0.033%

VectorBT CDLENGULFING with ATR Trailing Stop Results
Initial Capital: ₹100,000
Final Value: ₹9,885,820.23
Total Return: -1.14%
Total Trades: 9
Win Rate: 33.33%
Max Drawdown: 1.90%
Sharpe Ratio: -13.99
Profit Factor: 0.48
Average ATR Stop Distance: 0.36%

Trade Details:
Direction  Avg Entry Price  Avg Exit Price           PnL    Return Status
     Long      1583.500000     1577.400024 -45066.275091 -0.004508 Closed
     Long      1570.099976     1579.000000  49852.048161  0.005009 Closed
     Long      1579.599976     1586.000000  33936.882483  0.003393 Closed
     Long      1553.000000     1549.199951 -31142.612387 -0.003103 Closed
     Long      1545.300049     1542.099976 -27284.964024 -0.002727 Closed
     Long      1532.599976     1531.199951 -15667.503665 -0.001570 Closed
     Long      1530.000000     1522.400024 -56011.057925 -0.005623 Closed
     Long      1503.599976     1507.699951  20491.355399  0.002069 Closed
     Long      1511.400024    

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_13140/2798812985.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stop_distance = stop_distance.fillna(method='ffill').fillna(0)
/Users/hemank/Documents/github/.talib/lib/python3.12/site-packages/vectorbt/utils/datetime_.py:24: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  return pd.Timedelta(arg)


In [ ]:
import subprocess
import json
import yfinance as yf
import numpy as np
import pandas as pd
import joblib
import requests
from datetime import datetime, timedelta
import asyncio
import concurrent.futures
from typing import Dict, List, Tuple

class MultiAgentTradingSystem:
    def __init__(self, ollama_base_url="http://localhost:11434"):
        self.ollama_url = ollama_base_url
        self.agents = {
            "technical_analyst": "mistral:latest",
            "fundamental_analyst": "llama3:latest", 
            "sentiment_analyst": "mistral:latest",
            "risk_manager": "llama3:latest",
            "portfolio_manager": "mistral:latest"
        }
    
    def fetch_comprehensive_data(self, ticker="INFY.NS"):
        """Fetch comprehensive market data for analysis"""
        # Get intraday data
        intraday = yf.download(ticker, interval="5m", period="1d", progress=False)
        
        # Get historical data for context
        historical = yf.download(ticker, period="30d", progress=False)
        
        # Get stock info
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Calculate technical indicators
        def add_indicators(df):
            df['SMA5'] = df['Close'].rolling(5).mean()
            df['SMA20'] = df['Close'].rolling(20).mean()
            df['EMA12'] = df['Close'].ewm(span=12).mean()
            df['EMA26'] = df['Close'].ewm(span=26).mean()
            df['MACD'] = df['EMA12'] - df['EMA26']
            
            # RSI
            delta = df['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # Bollinger Bands
            sma20 = df['Close'].rolling(20).mean()
            std20 = df['Close'].rolling(20).std()
            df['BB_Upper'] = sma20 + (std20 * 2)
            df['BB_Lower'] = sma20 - (std20 * 2)
            
            return df
        
        intraday = add_indicators(intraday)
        historical = add_indicators(historical)
        
        return {
            "intraday": intraday,
            "historical": historical,
            "info": info,
            "current_price": float(intraday['Close'].iloc[-1]),
            "volume_avg": float(historical['Volume'].mean())
        }
    
    def query_agent(self, agent_name: str, prompt: str, model: str) -> str:
        """Query individual agent"""
        try:
            response = requests.post(
                f"{self.ollama_url}/api/generate",
                json={
                    "model": model,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.3,
                        "top_p": 0.9,
                        "num_predict": 500
                    }
                },
                timeout=45
            )
            
            if response.status_code == 200:
                return response.json()["response"]
            else:
                return f"Error: Agent {agent_name} failed (Status: {response.status_code})"
                
        except Exception as e:
            return f"Error querying {agent_name}: {str(e)}"
    
    def technical_analysis_agent(self, data: Dict) -> str:
        """Technical Analysis Agent"""
        latest = data["intraday"].iloc[-1]
        prev = data["intraday"].iloc[-2] if len(data["intraday"]) > 1 else latest
        
        prompt = f"""You are a Senior Technical Analyst specializing in Indian stock markets with 15+ years experience.

INFOSYS (INFY.NS) TECHNICAL DATA:
Current Price: ₹{latest['Close']:.2f}
Previous Price: ₹{prev['Close']:.2f}
Change: {((latest['Close'] - prev['Close']) / prev['Close'] * 100):.2f}%

MOVING AVERAGES:
- SMA5: ₹{latest['SMA5']:.2f}
- SMA20: ₹{latest['SMA20']:.2f}
- EMA12: ₹{latest['EMA12']:.2f}
- EMA26: ₹{latest['EMA26']:.2f}

MOMENTUM INDICATORS:
- RSI(14): {latest['RSI']:.1f}
- MACD: {latest['MACD']:.3f}

BOLLINGER BANDS:
- Upper: ₹{latest['BB_Upper']:.2f}
- Lower: ₹{latest['BB_Lower']:.2f}
- Current vs Bands: {'Above Upper' if latest['Close'] > latest['BB_Upper'] else 'Below Lower' if latest['Close'] < latest['BB_Lower'] else 'Within Bands'}

VOLUME:
- Current: {latest['Volume']:,.0f}
- 30-day Average: {data['volume_avg']:,.0f}
- Volume Ratio: {(latest['Volume'] / data['volume_avg']):.1f}x

Provide your technical analysis for INTRADAY TRADING with:
1. Signal: BUY/SELL/HOLD
2. Confidence: 1-10
3. Key technical levels (support/resistance)
4. Entry/exit strategy
5. Risk assessment"""

        return self.query_agent("technical_analyst", prompt, self.agents["technical_analyst"])
    
    def fundamental_analysis_agent(self, data: Dict) -> str:
        """Fundamental Analysis Agent"""
        info = data["info"]
        
        prompt = f"""You are a Senior Fundamental Analyst with expertise in IT sector analysis.

INFOSYS FUNDAMENTAL DATA:
Market Cap: ₹{info.get('marketCap', 'N/A'):,} Cr
P/E Ratio: {info.get('trailingPE', 'N/A')}
P/B Ratio: {info.get('priceToBook', 'N/A')}
ROE: {info.get('returnOnEquity', 'N/A')}
Debt-to-Equity: {info.get('debtToEquity', 'N/A')}
Revenue Growth: {info.get('revenueGrowth', 'N/A')}
Profit Margin: {info.get('profitMargins', 'N/A')}
Current Price: ₹{data['current_price']:.2f}

SECTOR: Information Technology Services
BUSINESS: Global consulting, technology, outsourcing solutions

Today's Date: {datetime.now().strftime('%Y-%m-%d')}

Consider:
- Q2 FY25 earnings season context
- IT sector trends and client spending
- Currency impact (USD-INR)
- Digital transformation demand
- Competition landscape

Provide fundamental perspective for INTRADAY trading:
1. Fair value assessment
2. Sector outlook impact
3. Any immediate fundamental catalysts
4. Risk factors to watch
5. Recommendation with rationale"""

        return self.query_agent("fundamental_analyst", prompt, self.agents["fundamental_analyst"])
    
    def sentiment_analysis_agent(self, data: Dict) -> str:
        """Market Sentiment Analysis Agent"""
        prompt = f"""You are a Market Sentiment Analyst tracking social media, news, and market psychology.

INFOSYS CURRENT CONTEXT (July 29, 2025):
Current Price: ₹{data['current_price']:.2f}
Recent Price Action: {((data['intraday']['Close'].iloc[-1] - data['intraday']['Open'].iloc[0]) / data['intraday']['Open'].iloc[0] * 100):.2f}% today

SENTIMENT FACTORS TO CONSIDER:
- IT sector investor sentiment
- Foreign institutional investor (FII) activity
- Domestic institutional investor (DII) trends
- Social media buzz around tech stocks
- Market fear/greed indicators
- Sector rotation patterns
- Global tech sentiment spillover

MARKET ENVIRONMENT:
- NSE/BSE overall sentiment
- FII/DII flows
- Global IT services demand
- Geopolitical factors affecting IT outsourcing

Analyze market sentiment for INTRADAY trading:
1. Overall sentiment: Bullish/Bearish/Neutral (1-10 scale)
2. Social sentiment indicators
3. Institutional activity impact
4. Retail investor behavior
5. Sentiment-driven price targets
6. Momentum sustainability assessment"""

        return self.query_agent("sentiment_analyst", prompt, self.agents["sentiment_analyst"])
    
    def risk_management_agent(self, data: Dict, xgb_signal: str, xgb_confidence: float) -> str:
        """Risk Management Agent"""
        latest = data["intraday"].iloc[-1]
        volatility = data["historical"]['Close'].pct_change().std() * np.sqrt(252) * 100
        
        prompt = f"""You are a Chief Risk Officer specializing in intraday trading risk management.

INFOSYS RISK PROFILE:
Current Price: ₹{data['current_price']:.2f}
Historical Volatility: {volatility:.1f}% (annualized)
Daily Range: ₹{data['intraday']['High'].max() - data['intraday']['Low'].min():.2f}
Current RSI: {latest['RSI']:.1f}

POSITION CONTEXT:
- ML Model Signal: {xgb_signal}
- ML Confidence: {xgb_confidence:.1%}
- Market Session: {'Pre-market' if datetime.now().hour < 9 else 'Regular' if 9 <= datetime.now().hour < 15.5 else 'Post-market'}
- Liquidity: High (Large cap stock)

RISK FACTORS:
- Intraday volatility spikes
- News/announcement risks
- Sector-wide movements
- Market circuit breaker risks
- Liquidity risks during lunch break

Provide comprehensive risk assessment for INTRADAY position:
1. Risk Rating: Low/Medium/High
2. Maximum position size recommendation (% of capital)
3. Stop-loss levels (multiple scenarios)
4. Take-profit targets
5. Time-based exit rules
6. Risk-adjusted return expectations
7. Hedging suggestions if needed
8. Market timing considerations"""

        return self.query_agent("risk_manager", prompt, self.agents["risk_manager"])
    
    def portfolio_management_agent(self, technical_rec: str, fundamental_rec: str, 
                                 sentiment_rec: str, risk_rec: str) -> str:
        """Portfolio Manager - Final Decision Agent"""
        prompt = f"""You are a Senior Portfolio Manager making the final trading decision based on team analysis.

TEAM RECOMMENDATIONS:

TECHNICAL ANALYST:
{technical_rec[:500]}...

FUNDAMENTAL ANALYST:
{fundamental_rec[:500]}...

SENTIMENT ANALYST:
{sentiment_rec[:500]}...

RISK MANAGER:
{risk_rec[:500]}...

PORTFOLIO CONTEXT:
- Capital Available: Assume moderate risk tolerance
- Time Horizon: Intraday only
- Market Conditions: Normal trading session
- Stock: INFOSYS (Large cap, high liquidity)

Synthesize all inputs and provide FINAL TRADING DECISION:

FORMAT YOUR RESPONSE AS:
FINAL DECISION: [BUY/NO BUY/WAIT]
CONVICTION: [High/Medium/Low]
POSITION SIZE: [% of available capital]
ENTRY STRATEGY: [Market/Limit order details]
STOP LOSS: [Specific price level]
TAKE PROFIT: [Target price(s)]
TIME HORIZON: [Expected holding period]
KEY REASONING: [2-3 bullet points summarizing decision rationale]
RISK LEVEL: [Low/Medium/High]"""

        return self.query_agent("portfolio_manager", prompt, self.agents["portfolio_manager"])

def setup_trading_agents():
    """Setup multi-agent trading framework"""
    setup_commands = [
        "# Install Ollama if not already installed",
        "curl -fsSL https://ollama.ai/install.sh | sh",
        "",
        "# Pull required models",
        "ollama pull mistral:latest",
        "ollama pull llama3:latest", 
        "",
        "# Start Ollama server (run in background)",
        "ollama serve &",
        "",
        "# Install Python dependencies",
        "pip install yfinance pandas numpy requests joblib scikit-learn xgboost"
    ]
    return setup_commands

def run_multi_agent_analysis(ticker="INFY.NS", xgb_model_path=None):
    """Run comprehensive multi-agent LLM analysis"""
    
    print("🚀 MULTI-AGENT TRADING SYSTEM ACTIVATED")
    print("=" * 60)
    
    # Initialize system
    trading_system = MultiAgentTradingSystem()
    
    # Fetch comprehensive data
    print("📊 Fetching comprehensive market data...")
    try:
        data = trading_system.fetch_comprehensive_data(ticker)
        print(f"✓ Data fetched successfully for {ticker}")
    except Exception as e:
        return f"Error fetching data: {e}"
    
    # Run XGBoost model if available
    xgb_signal = "NEUTRAL"
    xgb_confidence = 0.5
    
    if xgb_model_path:
        try:
            model = joblib.load(xgb_model_path)
            latest = data["intraday"].iloc[-1]
            X = np.array([[
                latest['Close'], latest['SMA5'], latest['SMA20'], 
                latest['RSI'], latest['Volume'], latest['MACD']
            ]])
            pred = model.predict(X)[0]
            prob = model.predict_proba(X)[0]
            xgb_signal = "BUY" if pred == 1 else "NO BUY"
            xgb_confidence = float(prob[1] if pred == 1 else prob[0])
            print(f"🤖 XGBoost Signal: {xgb_signal} ({xgb_confidence:.1%} confidence)")
        except:
            print("⚠️  XGBoost model not available, using technical indicators only")
    
    print("\n🧠 CONSULTING MULTI-AGENT TEAM...")
    print("-" * 40)
    
    # Run agents concurrently for faster execution
    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # Submit all agent tasks
            technical_future = executor.submit(trading_system.technical_analysis_agent, data)
            fundamental_future = executor.submit(trading_system.fundamental_analysis_agent, data)
            sentiment_future = executor.submit(trading_system.sentiment_analysis_agent, data)
            risk_future = executor.submit(trading_system.risk_management_agent, data, xgb_signal, xgb_confidence)
            
            # Collect results
            print("📈 Technical Analyst working...")
            technical_rec = technical_future.result()
            
            print("📊 Fundamental Analyst working...")
            fundamental_rec = fundamental_future.result()
            
            print("📱 Sentiment Analyst working...")
            sentiment_rec = sentiment_future.result()
            
            print("🛡️  Risk Manager working...")
            risk_rec = risk_future.result()
            
        # Final portfolio manager decision
        print("👔 Portfolio Manager synthesizing recommendations...")
        final_decision = trading_system.portfolio_management_agent(
            technical_rec, fundamental_rec, sentiment_rec, risk_rec
        )
        
        # Compile comprehensive report
        report = f"""
🎯 MULTI-AGENT TRADING ANALYSIS REPORT
{'='*60}
Stock: {ticker.upper()}
Analysis Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S IST')}
Current Price: ₹{data['current_price']:.2f}

📈 TECHNICAL ANALYSIS:
{technical_rec}

📊 FUNDAMENTAL ANALYSIS:
{fundamental_rec}

📱 SENTIMENT ANALYSIS:
{sentiment_rec}

🛡️  RISK MANAGEMENT:
{risk_rec}

👔 FINAL PORTFOLIO DECISION:
{final_decision}

{'='*60}
⚠️  DISCLAIMER: This is AI-generated analysis for educational purposes only.
   Always conduct your own research and consult financial advisors.
{'='*60}
"""
        
        return report
        
    except Exception as e:
        return f"Error running multi-agent analysis: {str(e)}"

# Usage example
if __name__ == "__main__":
    # Setup (run once)
    print("Setup commands:")
    for cmd in setup_trading_agents():
        print(cmd)
    
    print("\n" + "="*60)
    
    # Run analysis
    result = run_multi_agent_analysis("INFY.NS")
    print(result)
